# Behavioural cloning project

In [1]:
# Configure matlab to show graphics in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Change to 'data' when training on a GPU
PATH_TO_DATA = 'data/sample'

In [3]:
# Shape of the image
input_shape = (160, 320, 3)

In [4]:
def img_folder(data_folder):
    return '{}/IMG'.format(data_folder)

def path_driving_log(data_folder):
    return '{}/driving_log.csv'.format(data_folder)

In [5]:
datasets = ['train', 'test', 'valid']

In [6]:
def get_driving_log_dataframe(data_folder):
    driving_log_df = pd.read_csv(path_driving_log(data_folder))
    return driving_log_df

In [7]:
path_to_folders = dict(zip(datasets, map(lambda folder: '{0}/{1}'.format(PATH_TO_DATA, folder), datasets)))

In [8]:
path_to_folders

{'test': 'data/sample/test',
 'train': 'data/sample/train',
 'valid': 'data/sample/valid'}

In [9]:
from scipy.ndimage import imread
from os import listdir
from sklearn.utils import shuffle

def image_label_generator(data_folder, batch_size=2):
    driving_log_df = get_driving_log_dataframe(data_folder)
    number_of_examples = len(driving_log_df)
    image_columns = ['center', 'left', 'right']
    
    X_train = []
    y_train = []
    index_in_batch = 0
    batch_number = 0
    
    while True:
        for image_column in image_columns:
            image_series = driving_log_df[image_column]
            steering_series = driving_log_df['steering']
            for offset in range(0, number_of_examples, batch_size):
                X_train = []
                y_train = []

                end_of_batch = min(number_of_examples, offset + batch_size)

                for j in range(offset, end_of_batch):
                    image_filename = image_series[j].lstrip().rstrip()
                    image = imread('{0}/{1}'.format(data_folder, image_filename))
                    label = steering_series[j]
                    X_train.append(image)
                    y_train.append(label)
                    X_train, y_train = shuffle(X_train, y_train)

                yield np.array(X_train), np.array(y_train)

In [10]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten

Using TensorFlow backend.


In [11]:
model = Sequential()

In [12]:
model.add(Convolution2D(3, 1, 1, 
                          input_shape=input_shape, 
                          border_mode='same', 
                          activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(24, 5, 5,
                       activation='relu',
                       subsample=(2, 2)))
model.add(BatchNormalization())
model.add(Convolution2D(36, 5, 5,
                       activation='relu',
                       subsample=(2, 2)))
model.add(BatchNormalization())
model.add(Convolution2D(48, 5, 5,
                       activation='relu',
                       subsample=(2, 2)))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 160, 320, 3)   12          convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 160, 320, 3)   12          convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 78, 158, 24)   1824        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 78, 158, 24)   96          convolution2d_2[0][0]            
___________________________________________________________________________________________

In [45]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [46]:
model.compile(optimizer='adam', 
              loss='mse',
             metrics=['mean_squared_error'])

In [47]:
train_generator = image_label_generator(path_to_folders['train'])
valid_generator = image_label_generator(path_to_folders['valid'])

In [48]:
model.fit_generator(train_generator, 
                    samples_per_epoch=2, 
                    nb_epoch=10, 
                    validation_data=valid_generator,
                    nb_val_samples=1,
                    callbacks=callbacks_list)

Epoch 1/10
Epoch 00000: val_loss improved from inf to 0.00254, saving model to weights-improvement-00-0.00.hdf5
2/2 [==============================] - 12s - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.0025 - val_mean_squared_error: 0.0025
Epoch 2/10
Epoch 00001: val_loss did not improve
2/2 [==============================] - 0s - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0038 - val_mean_squared_error: 0.0038
Epoch 3/10
Epoch 00002: val_loss did not improve
2/2 [==============================] - 0s - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0025 - val_mean_squared_error: 0.0025
Epoch 4/10
Epoch 00003: val_loss did not improve
2/2 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0038 - val_mean_squared_error: 0.0038
Epoch 5/10
Epoch 00004: val_loss did not improve
2/2 [==============================] - 0s - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.0025 - val_mean_squared_error: 0.0025
E

In [59]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [15]:
X_val, y_val = next(image_label_generator(path_to_folders['train']))

In [16]:
model.predict(X_val)

array([[ 0.       ],
       [ 0.0610016]], dtype=float32)